<a href="https://colab.research.google.com/github/surajit93/open-university-video/blob/main/production_control_center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-darkgrid")

PERF_DB = "../data/performance.db"
IMPROV_DB = "../data/improvement_history.db"


In [ ]:
def load_topic_scores():
    try:
        with open("../data/topic_scores.json") as f:
            data = json.load(f)
        return pd.DataFrame(data)
    except:
        return pd.DataFrame({
            "topic": ["AI Regulation", "AI Collapse", "AI Jobs"],
            "score": [0.78, 0.65, 0.82],
            "cluster": ["policy", "fear", "economy"]
        })

df_topics = load_topic_scores()

plt.figure(figsize=(10,5))
sns.barplot(data=df_topics, x="topic", y="score", hue="cluster")
plt.title("Topic Rank Scores")
plt.xticks(rotation=30)
plt.show()

# Cluster dominance
cluster_avg = df_topics.groupby("cluster")["score"].mean().sort_values(ascending=False)
cluster_avg.plot(kind="bar", title="Cluster Dominance")
plt.show()


In [ ]:
def load_hooks():
    try:
        with open("../data/hook_candidates.json") as f:
            return pd.DataFrame(json.load(f))
    except:
        return pd.DataFrame({
            "hook": [
                "AI will replace 60% of jobs by 2030.",
                "The AI shift is already underway.",
                "Your job may disappear faster than you think.",
                "AI might redefine employment forever.",
                "The next 5 years could shock workers."
            ],
            "curiosity": [0.8, 0.6, 0.9, 0.7, 0.85],
            "emotion": [0.9, 0.5, 0.88, 0.75, 0.82],
            "specificity": [0.7, 0.4, 0.85, 0.6, 0.78]
        })

df_hooks = load_hooks()
df_hooks["composite"] = (
    df_hooks["curiosity"] * 0.4 +
    df_hooks["emotion"] * 0.4 +
    df_hooks["specificity"] * 0.2
)

df_hooks = df_hooks.sort_values("composite", ascending=False)
display(df_hooks.style.highlight_max(subset=["composite"]))


In [ ]:
# mock intensity per paragraph
intensity = [0.6, 0.7, 0.75, 0.72, 0.85, 0.9, 0.88]
timeline = range(len(intensity))

plt.figure(figsize=(8,4))
plt.plot(timeline, intensity, marker="o")
plt.title("Tension Curve")
plt.xlabel("Scene")
plt.ylabel("Intensity")
plt.axhline(y=0.7, linestyle="--")
plt.show()


In [ ]:
depth_data = {
    "numerical_density": 0.65,
    "named_entities": 0.7,
    "vague_penalty": 0.3
}

plt.bar(depth_data.keys(), depth_data.values())
plt.title("Depth Score Components")
plt.show()


In [ ]:
thumb_data = pd.DataFrame({
    "variant": ["A", "B", "C", "D", "E"],
    "contrast_score": [0.7, 0.9, 0.6, 0.85, 0.75],
    "emotion_score": [0.8, 0.85, 0.6, 0.9, 0.7]
})

display(thumb_data.style.highlight_max())


In [ ]:
retention = np.array([100, 92, 85, 83, 60, 58, 57, 55])
drop = np.diff(retention, prepend=100)

plt.figure(figsize=(8,3))
sns.heatmap([drop], cmap="Reds", annot=True)
plt.title("Retention Drop Heatmap")
plt.show()


In [ ]:
conn = sqlite3.connect(IMPROV_DB)
try:
    df_emotion = pd.read_sql("""
        SELECT emotion_tag, COUNT(*) as count
        FROM improvement_history
        GROUP BY emotion_tag
    """, conn)
except:
    df_emotion = pd.DataFrame({
        "emotion_tag": ["fear", "curiosity", "optimism"],
        "count": [12, 5, 3]
    })
conn.close()

plt.pie(df_emotion["count"], labels=df_emotion["emotion_tag"], autopct="%1.1f%%")
plt.title("Channel Emotional Mix")
plt.show()


In [ ]:
velocity = [10, 25, 50, 80, 110, 150]
hours = range(len(velocity))

plt.plot(hours, velocity)
plt.axhline(y=120, linestyle="--")
plt.title("Views/Hour Early Velocity")
plt.show()


In [ ]:
uploads = np.arange(1,101)
expected_views = uploads * 1500 * 1.02**uploads

plt.plot(uploads, expected_views)
plt.title("Expected Growth Projection (100 Uploads)")
plt.xlabel("Uploads")
plt.ylabel("Projected Views")
plt.show()


In [ ]:
from scripts.manual_override import ManualOverride

override = ManualOverride()
recent = [
    {"ctr": 2.5, "retention_30s": 28},
    {"ctr": 2.7, "retention_30s": 25},
    {"ctr": 2.9, "retention_30s": 29},
    {"ctr": 2.6, "retention_30s": 27},
    {"ctr": 2.4, "retention_30s": 26},
]

paused, reason = override.evaluate(recent)
print("Paused:", paused)
print("Reason:", reason)


In [ ]:
conn = sqlite3.connect(PERF_DB)
try:
    df_week = pd.read_sql("""
        SELECT
            AVG(ctr) as avg_ctr,
            AVG(retention_30s) as avg_retention,
            AVG(avg_view_duration) as avg_duration
        FROM video_metrics
        WHERE upload_date >= date('now','-7 days')
    """, conn)
except:
    df_week = pd.DataFrame({
        "avg_ctr":[5.2],
        "avg_retention":[48],
        "avg_duration":[4.2]
    })
conn.close()

display(df_week)
